# Load Libraries

In [6]:
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

In [7]:
import openai
import os
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import json

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

openai.api_key = os.environ.get("OPENAI_API_KEY")

# Define Function

In [26]:
def fetch_tesla_stock_data(start_date, end_date):
    # Define the ticker symbol for Tesla
    ticker_symbol = "TSLA"

    # Fetch historical data using yfinance
    tesla_stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

    return tesla_stock_data


def compute_monthly_metrics(stock_data):
    # Resample data to monthly frequency and compute monthly returns
    monthly_data = stock_data['Close'].resample('M').mean()
    monthly_returns = monthly_data.pct_change() * 100



    # Combine monthly returns and volatility into a DataFrame
    monthly_metrics = pd.DataFrame({
        'return': monthly_returns,
    })

    return monthly_metrics.dropna()

In [36]:
def response_to_df(response):
    json_response = json.loads(response)

    response_df = pd.DataFrame()
    for month in json_response.keys():
        month_df = pd.DataFrame(json_response[month])
        month_df['Month'] = month
        response_df = pd.concat([response_df, month_df],axis=0,ignore_index=True,sort=False)

    response_df = response_df[['Month','event','analysis','percentage_change','probability']].copy()
    return response_df

In [76]:
def compute_event_returns(event_df, train_df):
    
    event_df = event_df.copy()
    output = pd.DataFrame(columns=['month','return'])
    
    mean_return = train_df.tail(3)['return'].mean()
    event_df['percentage_change'] = event_df['percentage_change'].str.split("%").str[0].replace("",np.nan)
    event_df['percentage_change'] = event_df['percentage_change'].astype('float').fillna(mean_return)
    
    for indx, month in enumerate(event_df['Month'].unique()):
        month_df = event_df[event_df['Month'] == month]
        weighted_return = month_df['percentage_change'].mul(month_df['probability']).sum()/(month_df['probability'].sum())
        output.loc[indx,['month','return']] = [month,weighted_return]  
    
    return output

# Get Dataset

In [9]:
end_date = datetime.datetime(2021, 12, 31)
start_date = end_date - datetime.timedelta(days=5 * 365)

In [27]:
tesla_data = fetch_tesla_stock_data(start_date, end_date)
tesla_data.head(3)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,14.324000,14.688667,14.064000,14.466000,14.466000,88849500
2017-01-04,14.316667,15.200000,14.287333,15.132667,15.132667,168202500
2017-01-05,15.094667,15.165333,14.796667,15.116667,15.116667,88675500


In [28]:
# define compute monthly return and volatility
df =compute_monthly_metrics(tesla_data)
df.head(5)

,return
Date,
2017-02-28,10.191585
2017-03-31,-2.106293
2017-04-30,18.051799
2017-05-31,3.860655
2017-06-30,15.568971


In [29]:
train_df = df[:-3].copy()
test_df = df[-3:].copy()

In [30]:
train_df.tail(3)

,return
Date,
2021-07-31,5.138652
2021-08-31,6.995295
2021-09-30,6.906724


In [31]:
test_df

,return
Date,
2021-10-31,16.499273
2021-11-30,27.587361
2021-12-31,-9.242450


# Prompt Engineering

In [32]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Zero Shot Prompting

In [104]:
prompt = f"""
Context:
As a seasoned financial analyst with a deep understanding of finance, economics,
geopolitics, competitor landscape, and regulatory environments, you are tasked 
with analyzing the key events that can impact the Tesla's (TSLA) stock price 
from October 2021 to December 2021.

Objective:

Generate a list of 5 major events, along with their analysis and 
percentage impact on TSLA's monthly stock price return. 
Carefully estimate the percentage impact based on the historical knowlege of Tesla stock price.
There can also be "no_event" if no significant event occurred.
For "no_event", the percentage impact should be blank.
After generating events for each month, accruately estimate the probabilities based on the historical evidence 


The output should be in JSON format with the following keys:

month: October 2021 to December 2021
event: Description of the event. 
analysis: Detailed explanation of how the event influenced TSLA's stock price. 
Consider factors like sentiment, market reaction, and news coverage.
accurate percentage_change: Estimated percentage change in TSLA's monthly stock price 
return due to the event based on the historical and forecasted news.
Use positive values for increases and negative values for decreases.
probability: 
The accurate probability of the event occurring, ranging 
from 0.0 (impossible) to 1.0 (certain).

Information Available:

Access to financial data, news articles, market reports, 
and economic indicators up to September 2021 and forecast reports for next three months.

Consider the following relevant factors:
Global economic climate (e.g., inflation, interest rates, GDP growth)
Tesla-specific news and announcements 
(e.g., deliveries, earnings, product launches)
expected Competitor actions and news (e.g., new EV releases, market share changes)
expected Geopolitical events and their potential impact on energy markets and 
Tesla's supply chain Regulatory changes affecting the EV industry

Output:

JSON files with keys month, event, analysis, percentage_change and probability.
Ensure the sum of event probabilities for each month equals 1.0.
Prioritize events with the highest potential impact on TSLA's stock price.

Note:
Feel free to include additional events or insights beyond the specified criteria.
Be creative and demonstrate your understanding of financial analysis 
and event-driven stock price movements.
"""
zero_shot_response = get_completion(prompt)
print(zero_shot_response)

{
  "October 2021": [
    {
      "event": "Tesla Q3 Earnings Report",
      "analysis": "Tesla's quarterly earnings reports have historically had a significant impact on its stock price. Positive earnings results, indicating strong revenue and profit growth, tend to drive the stock price higher. Conversely, negative earnings results can lead to a decline in the stock price. Investors closely watch Tesla's earnings reports for insights into the company's financial performance and future prospects.",
      "percentage_change": "5%",
      "probability": 0.8
    },
    {
      "event": "Global Chip Shortage",
      "analysis": "The ongoing global chip shortage has been affecting various industries, including the automotive sector. As Tesla heavily relies on semiconductors for its vehicles, any further disruptions in the chip supply chain could impact Tesla's production and delivery numbers. This could potentially lead to a decrease in Tesla's stock price as investors may become concerned

In [105]:
zero_shot_df = response_to_df(zero_shot_response)
zero_shot_df

,Month,event,analysis,percentage_change,probability
0,October 2021,Tesla Q3 Earnings Report,"Tesla's quarterly earnings reports have historically had a significant impact on its stock price. Positive earnings results, indicating strong revenue and profit growth, tend to drive the stock price higher. Conversely, negative earnings results can lead to a decline in the stock price. Investors closely watch Tesla's earnings reports for insights into the company's financial performance and future prospects.",5%,0.8
1,October 2021,Global Chip Shortage,"The ongoing global chip shortage has been affecting various industries, including the automotive sector. As Tesla heavily relies on semiconductors for its vehicles, any further disruptions in the chip supply chain could impact Tesla's production and delivery numbers. This could potentially lead to a decrease in Tesla's stock price as investors may become concerned about the company's ability to meet demand.",-3%,0.6
2,October 2021,Regulatory Changes in EV Industry,"Regulatory changes, such as new emissions standards or government incentives for electric vehicles, can significantly impact the EV industry and Tesla's market position. Positive regulatory changes, such as increased subsidies or stricter emissions regulations favoring EVs, could drive up Tesla's stock price as it benefits from a supportive regulatory environment. Conversely, negative regulatory changes could have a negative impact on Tesla's stock price.",2%,0.4
3,October 2021,Competitor Product Launch,"The launch of new electric vehicle models by Tesla's competitors can impact Tesla's market share and investor sentiment. If a competitor releases a highly anticipated and well-received EV, it could potentially attract customers away from Tesla and lead to a decrease in Tesla's stock price. On the other hand, if Tesla's competitors face challenges or receive negative reviews for their new models, it could benefit Tesla and result in an increase in its stock price.",1%,0.3
4,October 2021,Geopolitical Tensions Impacting Energy Markets,"Geopolitical tensions, such as conflicts in oil-producing regions or disruptions in energy supply chains, can impact energy markets and subsequently affect Tesla's stock price. If geopolitical events lead to a significant increase in oil prices or disruptions in the supply of critical materials for EV production, it could drive up Tesla's stock price as investors anticipate increased demand for electric vehicles. Conversely, geopolitical events that stabilize energy markets or favor traditional fossil fuel industries could have a negative impact on Tesla's stock price.",2%,0.2
5,November 2021,Tesla Cybertruck Production Update,"Tesla's Cybertruck has generated significant interest and anticipation among consumers and investors. Any updates on the production timeline, manufacturing progress, or pre-order numbers could impact Tesla's stock price. Positive updates indicating progress and strong demand could drive up the stock price, while delays or negative updates could lead to a decrease in the stock price.",3%,0.7
6,November 2021,Inflation Concerns,"Inflation concerns can impact the overall stock market and investor sentiment. Higher inflation rates can lead to increased borrowing costs, reduced consumer spending, and potential economic slowdown. As Tesla operates in the consumer discretionary sector, any signs of rising inflation and its potential impact on consumer purchasing power could negatively affect Tesla's stock price.",-2%,0.5
7,November 2021,Tesla Battery Day,"Tesla's Battery Day is an event where the company unveils its latest advancements in battery technology. Positive announcements regarding improved battery performance, cost reductions, or breakthroughs in energy storage could have a positive impact on Tesla's stock price. Conversely, if the announcements fail to meet expectations or lack significant advancements, it could lead to a decrease in the stock price.",4%,0.3
8,November 2

In [106]:
zero_shot_return = compute_event_returns(zero_shot_df, train_df)
zero_shot_return

,month,return
0,October 2021,1.608696
1,November 2021,1.55
2,December 2021,1.986723


In [107]:
zero_shot_df.to_csv("zero_shot_response.csv",index=False)
zero_shot_return.to_csv("zero_shot_return.csv",index=False)


In [120]:
actual_return = test_df.reset_index().copy()
actual_return['month'] = actual_return['Date'].dt.strftime("%B %Y")
actual_return.drop(['Date'],axis=1,inplace=True)
actual_return.rename({'return':'Actual Returns'}, axis=1, inplace=True)
actual_return = actual_return.merge(zero_shot_return, how='left', on='month')
actual_return.rename({'return':'Expected Returns', 'month':"Month"}, axis=1, inplace=True)
actual_return = actual_return[['Month','Actual Returns','Expected Returns']]
actual_return

,Month,Actual Returns,Expected Returns
0,October 2021,16.499273,1.608696
1,November 2021,27.587361,1.55
2,December 2021,-9.242450,1.986723


In [121]:
actual_return.to_csv("zero_shot_error.csv",index=False)


In [123]:
train_df.tail(3)

,return
Date,
2021-07-31,5.138652
2021-08-31,6.995295
2021-09-30,6.906724


# Few Shot Prompting

In [127]:
prompt = f"""
Generate 10 potential events that may impact Tesla's monthly stock price return from October to December 2021, including:

Event: A concise description of the event, including relevant details.
Analysis: A breakdown of the event's potential impact on Tesla, considering both positive and negative factors.
Estimated Impact: A percentage rangeof the potential change in Tesla's stock price return due to this event.
Estimated Probability: A percentage of the likelihood of this event occurring.
Provide the response in JSON format

Examples:

Month: July 2021
Event 1: Strong fundamentals
Analysis:  Better Q1 results and Q2 deliveries, and strong sales in China.
Estimated Impact: 5%
Estimated Probability: 0.6

Month: July 2021
Event 2: Disruption advantage
Analysis: COVID-19 potentially solidifying Tesla's position as the leading electric and autonomous driving player, while some mainstream automakers slow down investments
Estimated Impact: 15%
Estimated Probability: 0.95

Month: August 2021
Event 1: Strong self-driving software sales
Analysis: Despite negative news, sales continue to grow, indicating high demand for the technology.
Estimated Impact: 10%
Estimated Probability: 0.8

Month: September 2021
Event 1: Analyst Expectation
Analysis: 63% chance of stock rise in the next month
Estimated Impact: 12%
Estimated Probability: 0.75

"""
few_shot_response = get_completion(prompt)
print(few_shot_response)

{
  "October 2021": [
    {
      "Event": "Battery Day announcements",
      "Analysis": "Tesla unveils new advancements in battery technology, potentially improving the range and performance of their vehicles.",
      "Estimated Impact": "8-10%",
      "Estimated Probability": "0.7"
    },
    {
      "Event": "Supply chain disruptions",
      "Analysis": "Global supply chain issues impact Tesla's production and delivery capabilities, leading to potential delays and lower sales.",
      "Estimated Impact": "-5 to -8%",
      "Estimated Probability": "0.4"
    },
    {
      "Event": "Regulatory changes",
      "Analysis": "New regulations or policies related to electric vehicles and renewable energy could either benefit or hinder Tesla's operations and market demand.",
      "Estimated Impact": "-3 to 5%",
      "Estimated Probability": "0.6"
    },
    {
      "Event": "Competition intensifies",
      "Analysis": "Rival automakers launch new electric vehicle models, increasing compe

In [133]:
few_shot_json = json.loads(few_shot_response)
few_shot_df = pd.DataFrame()
for month in few_shot_json.keys():
    month_df = pd.DataFrame(few_shot_json[month])
    month_df['Month'] = month
    few_shot_df = pd.concat([few_shot_df, month_df],axis=0,ignore_index=True,sort=False)

In [135]:
few_shot_df = few_shot_df[['Month','Event','Analysis','Estimated Impact','Estimated Probability']].copy()
few_shot_df

,Month,Event,Analysis,Estimated Impact,Estimated Probability
0,October 2021,Battery Day announcements,"Tesla unveils new advancements in battery technology, potentially improving the range and performance of their vehicles.",8-10%,0.7
1,October 2021,Supply chain disruptions,"Global supply chain issues impact Tesla's production and delivery capabilities, leading to potential delays and lower sales.",-5 to -8%,0.4
2,October 2021,Regulatory changes,New regulations or policies related to electric vehicles and renewable energy could either benefit or hinder Tesla's operations and market demand.,-3 to 5%,0.6
3,October 2021,Competition intensifies,"Rival automakers launch new electric vehicle models, increasing competition in the market and potentially impacting Tesla's market share.",-5 to -7%,0.8
4,October 2021,Earnings report,"Tesla releases its quarterly earnings report, which could exceed or fall short of market expectations, influencing investor sentiment.",-5 to 10%,0.9
5,November 2021,Infrastructure bill passage,"The passing of a significant infrastructure bill includes provisions that support electric vehicle adoption, benefiting Tesla's market outlook.",5-8%,0.6
6,November 2021,Battery supply constraints,"Shortages in battery supply impact Tesla's production capacity, leading to potential production cuts and lower sales.",-3 to -6%,0.5
7,November 2021,New product announcement,"Tesla unveils a new product or service, generating excitement and potential increased demand for the company's offerings.",3-6%,0.7
8,November 2021,Macroeconomic factors,"Changes in interest rates, inflation, or overall market sentiment could influence investor behavior and impact Tesla's stock price.",-4 to 4%,0.8
9,November 2021,Partnership announcement,"Tesla announces a strategic partnership with another company, potentially opening new market opportunities or enhancing existing operations.",2-5%,0.6


In [136]:
few_shot_df.to_csv("few_shot_response.csv",index=False)


In [141]:
few_shot_df['Estimated Impact'] = few_shot_df['Estimated Impact'].str.replace("-","to").str.split("to").str[-1].str.split("%").str[0]

In [144]:
few_shot_return = pd.DataFrame(columns=['Month','EXpected Return'])
for indx, month in enumerate(few_shot_df['Month'].unique()):
    month_df = few_shot_df[few_shot_df['Month']==month].copy()
    month_df['Estimated Probability'] = month_df['Estimated Probability'].astype(float)
    month_df['Estimated Impact'] = month_df['Estimated Impact'].astype(float)
    weighted_return = month_df['Estimated Impact'].mul(month_df['Estimated Probability']).sum()/month_df['Estimated Probability'].sum()
    few_shot_return.loc[indx,['Month','EXpected Return']] = [month,weighted_return]

In [145]:
few_shot_return

,Month,EXpected Return
0,October 2021,8.176471
1,November 2021,5.6875
2,December 2021,5.657143


In [150]:
actual_return.drop(['Expected Returns'], axis=1).merge(few_shot_return, how='left', on='Month')

,Month,Actual Returns,EXpected Return
0,October 2021,16.499273,8.176471
1,November 2021,27.587361,5.6875
2,December 2021,-9.242450,5.657143


# Chain of reasoning Prompt

In [164]:
prompt = f"""
Generate 10 potential events that may impact Tesla's monthly stock price return from October to December 2021, including:

Event: A concise description of the event, including relevant details.
Analysis: A breakdown of the event's potential impact on Tesla, considering both positive and negative factors.
Estimated Impact: A percentage rangeof the potential change in Tesla's stock price return due to this event.
Estimated Probability: A percentage of the likelihood of this event occurring.
Provide the response in JSON format

Context:

Analysts forecast significant growth for Tesla's stock price in October-December 2021. This coincides with:
Strong Q3 2021 performance: Surpassing both Wall Street and Estimize revenue projections, indicating robust financial health and potential for continued growth.
Positive global economic backdrop: Vaccination progress, government stimulus, and consumer confidence recovery fueled a strong economic rebound, creating a favorable environment for investments.
Record quarterly deliveries in Q4 2021: Highlighting Tesla's dominant position in the EV market and exceeding delivery targets, further solidifying investor confidence.
Positive analyst sentiment: Wall Street remained bullish on Tesla with Buy ratings and optimistic price targets, despite near-term concerns.
Scenarios:

Scenario 1: Exceeding Forecasts, Sustained Momentum (October-December 2021)

Reasoning:
Building on Q3 2021 success: Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge. (Impact: High, Probability: High)
Cybertruck anticipation: The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors. (Impact: Medium, Probability: High)
Ongoing economic recovery: Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla. (Impact: Medium, Probability: Moderate)
Positive media coverage and analyst upgrades: Continued positive press and upward revisions from analysts could further amplify the rally. (Impact: Medium, Probability: Moderate)
Outcomes:
Stock price surpasses analyst forecasts across October-December.
Increased institutional buying and strong retail investor participation.
Record trading volume and potential short-squeeze scenarios.
Scenario 2: Market Correction Despite Positive Fundamentals (Post-December 2021)

Reasoning:
Profit-taking after significant gains: A rally throughout October-December could trigger profit-taking and selling pressure from early investors, causing a temporary dip. (Impact: High, Probability: Moderate)
Missed Cybertruck expectations (beyond revised projections): A significant shortfall in Cybertruck deliveries could dampen investor enthusiasm and lead to a more substantial correction. (Impact: Medium, Probability: Low)
Broader market volatility: Unexpected events or negative news in the global economy could spark a general market sell-off, impacting Tesla alongside other stocks. (Impact: High, Probability: Moderate)
Competition in the EV market: Continued rise of rival EV players could pose a long-term threat to Tesla's market dominance and influence future valuation. (Impact: Medium, Probability: Moderate)
Outcomes:
Potential stock price dip after December, despite exceeding initial forecasts.
Increased analyst caution and potential downward target revisions.
Short-term volatility and heightened risk of bearish sentiment.

Output:

JSON files with keys month, event, analysis, percentage_change and probability.
Ensure the sum of event probabilities for each month equals 1.0.
Prioritize events with the highest potential impact on TSLA's stock price.
"""
reason_response = get_completion(prompt)
print(reason_response)

{
  "October": [
    {
      "event": "Exceeding Forecasts, Sustained Momentum",
      "analysis": "Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.",
      "percentage_change": "5-10%",
      "probability": "70%"
    },
    {
      "event": "Cybertruck anticipation",
      "analysis": "The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",
      "percentage_change": "3-7%",
      "probability": "80%"
    },
    {
      "event": "Ongoing economic recovery",
      "analysis": "Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",
      "percentage_change": "2-5%",
      "probability": "50%"
    },
    {
      "event": "Positive media coverage and analyst upgrades",
      "analysis": "Continued positive press and upward revisions from analysts coul

In [165]:
reason_json = json.loads(reason_response)
reason_shot_df = pd.DataFrame()
for month in reason_json.keys():
    month_df = pd.DataFrame(reason_json[month])
    month_df['Month'] = month
    reason_shot_df = pd.concat([reason_shot_df, month_df],axis=0,ignore_index=True,sort=False)

In [170]:
reason_shot_df =reason_shot_df[['Month','event', 'analysis', 'percentage_change', 'probability']].copy()
reason_shot_df

,Month,event,analysis,percentage_change,probability
0,October,"Exceeding Forecasts, Sustained Momentum",Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.,5-10%,70%
1,October,Cybertruck anticipation,"The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",3-7%,80%
2,October,Ongoing economic recovery,"Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",2-5%,50%
3,October,Positive media coverage and analyst upgrades,Continued positive press and upward revisions from analysts could further amplify the rally.,2-4%,50%
4,November,"Exceeding Forecasts, Sustained Momentum",Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.,5-10%,70%
5,November,Cybertruck anticipation,"The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",3-7%,80%
6,November,Ongoing economic recovery,"Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",2-5%,50%
7,November,Positive media coverage and analyst upgrades,Continued positive press and upward revisions from analysts could further amplify the rally.,2-4%,50%
8,December,Market Correction Despite Positive Fundamentals,"A rally throughout October-December could trigger profit-taking and selling pressure from early investors, causing a temporary dip.",-5% to -10%,60%
9,December,Missed Cybertruck expectations (beyond revised projections),A significant shortfall in Cybertruck deliveries could dampen investor enthusiasm and lead to a more substantial correction.,-3% to -7%,20%


In [173]:
reason_shot_df['Month'] = reason_shot_df['Month'] + " 2021"
reason_shot_df

,Month,event,analysis,percentage_change,probability
0,October 2021,"Exceeding Forecasts, Sustained Momentum",Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.,5-10%,70%
1,October 2021,Cybertruck anticipation,"The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",3-7%,80%
2,October 2021,Ongoing economic recovery,"Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",2-5%,50%
3,October 2021,Positive media coverage and analyst upgrades,Continued positive press and upward revisions from analysts could further amplify the rally.,2-4%,50%
4,November 2021,"Exceeding Forecasts, Sustained Momentum",Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.,5-10%,70%
5,November 2021,Cybertruck anticipation,"The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",3-7%,80%
6,November 2021,Ongoing economic recovery,"Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",2-5%,50%
7,November 2021,Positive media coverage and analyst upgrades,Continued positive press and upward revisions from analysts could further amplify the rally.,2-4%,50%
8,December 2021,Market Correction Despite Positive Fundamentals,"A rally throughout October-December could trigger profit-taking and selling pressure from early investors, causing a temporary dip.",-5% to -10%,60%
9,December 2021,Missed Cybertruck expectations (beyond revised projections),A significant shortfall in Cybertruck deliveries could dampen investor enthusiasm and lead to a more substantial correction.,-3% to -7%,20%


In [174]:
reason_shot_df.to_csv("chain_of_reasoning_response.csv",index=False)


In [176]:
reason_shot_df['percentage_change'] = reason_shot_df['percentage_change'].str.replace("-","to").str.split("to").str[-1].str.split("%").str[0]

In [177]:
reason_shot_df

,Month,event,analysis,percentage_change,probability
0,October 2021,"Exceeding Forecasts, Sustained Momentum",Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.,10,70%
1,October 2021,Cybertruck anticipation,"The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",7,80%
2,October 2021,Ongoing economic recovery,"Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",5,50%
3,October 2021,Positive media coverage and analyst upgrades,Continued positive press and upward revisions from analysts could further amplify the rally.,4,50%
4,November 2021,"Exceeding Forecasts, Sustained Momentum",Continued exceeding of analyst expectations throughout the period could further elevate sentiment and fuel a sustained stock price surge.,10,70%
5,November 2021,Cybertruck anticipation,"The much-awaited Cybertruck launch, even if meeting revised delivery projections, could generate significant excitement and attract new investors.",7,80%
6,November 2021,Ongoing economic recovery,"Sustained strong macro-economic conditions could maintain a bullish environment for all assets, including Tesla.",5,50%
7,November 2021,Positive media coverage and analyst upgrades,Continued positive press and upward revisions from analysts could further amplify the rally.,4,50%
8,December 2021,Market Correction Despite Positive Fundamentals,"A rally throughout October-December could trigger profit-taking and selling pressure from early investors, causing a temporary dip.",10,60%
9,December 2021,Missed Cybertruck expectations (beyond revised projections),A significant shortfall in Cybertruck deliveries could dampen investor enthusiasm and lead to a more substantial correction.,7,20%


In [182]:
reason_return = pd.DataFrame(columns=['Month','Expected Return'])
for indx, month in enumerate(reason_shot_df['Month'].unique()):
    month_df = reason_shot_df[reason_shot_df['Month']==month].copy()
    month_df['probability']  = month_df['probability'].str.split("%").str[0]
    month_df['probability'] = month_df['probability'].astype(float)/100
    month_df['percentage_change'] = month_df['percentage_change'].astype(float)
    weighted_return = month_df['percentage_change'].mul(month_df['probability']).sum()/month_df['probability'].sum()
    reason_return.loc[indx,['Month','Expected Return']] = [month,weighted_return]

In [183]:
reason_return

,Month,Expected Return
0,October 2021,6.84
1,November 2021,6.84
2,December 2021,8.375


In [185]:
actual_return.drop(['Expected Returns'], axis=1).merge(reason_return, how='left', on='Month').to_clipboard()